# Simulations QCQP - equivalent QP

<img src="http://media.giphy.com/media/elvaNxcznR8Fa/giphy.gif"  width="200">


In [ ]:
#http://www.cvxpy.org/
#to solve convex optimization pb
import cvxpy as cvx

In [ ]:
#https://github.com/cvxgrp/qcqp
#to solve nonconvex QCQP pb
from qcqp import *

In [ ]:
#other modules
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import randn

## Plan :

   [1. Data simulation](#1)

   [2. "isIncluded" algorithms](#2)  
   
   [3. QP vs. QCQP](#3) 
   
   [4. Other examples](#4)  

<a id="1"></a> 
 
# 1. Data simulation

Data simulation procedure :

Generate a random set of balls I and U (for Intersection and Union)

INPUT : 

epsilon = overlap parameter  
n = dimension    
p = number of intersection balls  
q = number of union balls  
diff = ratio of standard deviations for random values of centers between union and intersection balls

OUTPUT : set of balls for intersection (cI,RI) and set of balls for exclusion (cU,RU) 
with cI and cR center points, RI and RU radii.

In [ ]:
def dataGenerator(epsilon, n, p, q, diff):
    #initial ball
    rand = np.random.normal(0, 10, n)
    cIU = rand.reshape((1, n))
    RIU = np.sqrt((cIU*cIU).sum(axis=1)) + epsilon/2
    
    i = 0   
    while i < p+q:
        if i < p:
            rand = np.random.normal(0, 10, n)
            c = rand.reshape((1, n))
            R = np.sqrt((c*c).sum(axis=1)) + epsilon/2
        if i >= p:
            #sigma = 10*diff and R = distance to 0 + epsilon/2
            rand = np.random.normal(0, 10*diff, n)
            c = rand.reshape((1, n))
            R = np.sqrt((c*c).sum(axis=1)) + epsilon
        concat = True
        
        for j in range(0,cIU.shape[0]):
            dist = np.linalg.norm((cIU[j] - c), ord = 1)
            if dist < abs(RIU[j]-R): #if inclusion
                concat = False
        
        if concat == True:
            cIU = np.concatenate((cIU, c), axis = 0)
            RIU = np.concatenate((RIU, R), axis = 0)
            i = i + 1
    cU = cIU[p+1:p+q+1,]
    cI = cIU[0:p,] 
    RU = RIU[p+1:p+q+1]
    RI = RIU[0:p]     
    return(cI,RI,cU,RU)

Topology of the simulation: study of the intersections and inclusions between balls. If all constraints are verified, we get a matrix of ones.

In [ ]:
def topology(cI,RI,cU,RU,n,p,q):
    c = np.concatenate((cI, cU), axis=0)
    R = np.concatenate((RI, RU), axis=0)
    response = np.ones((p+q, p+q))
    for i in range(0,p+q):
        for j in range(0,p+q):
            dist = np.sqrt(sum((c[i]-c[j])*(c[i]-c[j])))
            if dist > R[i] + R[j] : 
                response[i,j] = 0
            if dist < R[j] - R[i] : 
                response[i,j] = 2
            if dist < R[i] - R[j] : 
                response[i,j] = 3
                
    return response


Plot the simulated data in 2D case

In [ ]:
def plot2d(cI,RI,cU,RU,p,q, scale = 30):
    fig, ax = plt.subplots() 
    ax.set_xlim((-scale, scale))
    ax.set_ylim((-scale, scale))

    for i in range(0, q):
        ax.add_artist(plt.Circle(list(cU[i]), RU[i], color='r',  alpha=1))
    for i in range(0, p):
        ax.add_artist(plt.Circle(list(cI[i]), RI[i], color='b', alpha=.15))

### An example

In [ ]:
epsilon = 5
diff = 3
#n = nb of varialbles
n = 2
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 3
(cI, RI, cU, RU) = dataGenerator(epsilon, n, p, q, diff)

In [ ]:
# A matrix filled by ones
topology(cI, RI, cU, RU, n, p, q)

In [ ]:
plot2d(cI,RI,cU,RU,p,q)

<img src="http://media.giphy.com/media/bvARFeSkNzIm4/giphy.gif"  width="300">

<a id="2"></a> 
 
# 2. "isIncluded" algorithms

## QCQP

Function "isIncluded" with QCQP algorithm (the naive algorithm 1.1)

In [ ]:
def IsIncluded_QCQP(cI,RI,cU,RU,n,p,q,i=0):
    myReturn = [False,q-i] #response, nb of loop

    x = cvx.Variable(n)
    while i < q: 
        objective = -cvx.sum_squares(x - cU[i])
        constraints  = [cvx.sum_squares(x - cI[0]) <= RI[0]**2]
        for k in range(1, p):
            constraints = constraints + [cvx.sum_squares(x - cI[k]) <= RI[k]**2]
        for l in range(0, i):
            constraints = constraints + [cvx.sum_squares(x - cU[l]) >= RU[l]**2]

        prob = cvx.Problem(cvx.Minimize(objective), constraints)  
        qcqp = QCQP(prob)
        qcqp.suggest(RANDOM) 
        f_cd = qcqp.improve(ADMM) 
        #f_cd = qcqp.improve(COORD_DESCENT) 
        if -f_cd[0] <= RU[i]**2: 
            myReturn[0] = True
            myReturn[1] = i + 1
            i = q  
        i = i+1
        
    return myReturn

Test IsIncluded_QCQP

In [ ]:
n = 2; p = 3; q = 3; diff =1; epsilon = 10
(cI, RI, cU, RU)= dataGenerator(epsilon, n, p, q, diff)
IsIncluded_QCQP(cI,RI,cU,RU,n,p,q)

In [ ]:
plot2d(cI,RI,cU,RU,p,q)

The parameter i can be chosen equal to q-1 to solve only the last $P_0(q)$ problem.

Function "isIncluded" using plot 2D

In [ ]:
def IsIncluded_PLOT2d(cI,RI,cU,RU,p,q):
    h = .1  # step size in the mesh
    RR = min(RI)
    s = np.argmin(RI)
    x_min, x_max =  cI[s,0] - RR, cI[s,0] + RR
    y_min, y_max = cI[s,1] - RR, cI[s,1] + RR
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    i = 0
    while i < len(xx):
        j = 0
        while j < len(yy):
            response = True
            for k in range(0,p):
                if((xx[i,j] - cI[k,0])**2 + (yy[i,j] - cI[k,1])**2 > RI[k]**2):
                    response = False
                    k = p
            if response == True:
                for l in range(0,q):
                    if((xx[i,j] - cU[l,0])**2 + (yy[i,j] - cU[l,1])**2 <= RU[l]**2):
                        response = False
                        l = q

            if response == True:
                i = len(xx)
                j = len(yy)
            j= j + 1
        i=i + 1
    return not response

Function "isIncluded" with QP algorithm

In [ ]:
from cvxopt import matrix, solvers #to solve convex optimization pb
import cdd #to solve vertex enumeration
import mosek

In [ ]:
def IsIncluded_QP(cI, RI, cU, RU, n, p, q):
    myReturn = [True, q, 0] #response, nb of loop, nb of vertex enum
    
    i = 0
    while i < q: 
        #Create constraints G*x <= h
        cU2 = np.delete(cU, (i), axis=0)
        RU2 = np.delete(RU, (i), axis=0)
        h1 = RI**2 - RU[i]**2 + sum(cU[i]**2) - (cI**2).sum(axis=1)
        h2 = RU[i]**2 - RU2**2 - sum(cU[i]**2) + (cU2**2).sum(axis=1)
        h = np.concatenate((h1,h2), axis=0)
        G1 = 2*(cU[i] - cI)
        G2 = 2*(cU2 - cU[i])
        G = np.matrix(np.concatenate((G1,G2), axis=0))

        # Create variable.
        x = cvx.Variable(n)
        # Pb solution.    
        constraints = [G*x <= h]
        obj = cvx.Minimize(cvx.sum_squares(x-cU[i]))
        prob = cvx.Problem(obj, constraints)
        
        if prob.solve() <RU[i]**2: #Study of the intersection circle - polyhedron -> vertex enumeration 
            myReturn[2] = myReturn[2] + 1
            hh = h.reshape(G.shape[0], 1) 
            m = np.asarray(np.concatenate((hh,-G), axis=1))
            mat = cdd.Matrix(m, number_type='float')
            mat.rep_type = cdd.RepType.INEQUALITY
            poly = cdd.Polyhedron(mat)
            result = np.array(poly.get_generators())
            
            if result.shape[0] > 0:
                if (all(result[:,0] == 1) == True):#polyhedron = polytope. test each edge
                    for j in range(0,result.shape[0]):
                        if (sum((result[j,1:]-cU[i])**2) > RU[i]**2):#vertex outside the ball
                            myReturn[0] = False
                else:
                    myReturn[0] = False #unbounded polyhedron   
        if myReturn[0] == False:
            myReturn[1] = i + 1
            i = q
        
        i = i+1 
    
    if myReturn[0] == True: #case: partial U intersection I = emptyset. test a point
        objective = cvx.sum_squares(x-cI[p-1])
        constraints  = [cvx.sum_squares(x-cI[0]) <= RI[0]**2]
        for k in range(1, p-1):
            constraints = constraints + [cvx.sum_squares(x-cI[k]) <= RI[k]**2]
        probINT = cvx.Problem(cvx.Minimize(objective), constraints)  
        qcqp = QCQP(probINT)
        qcqp.suggest(RANDOM)    
        f_cd = qcqp.improve(COORD_DESCENT)
        value = matrix(x.value.reshape(1,n))
        s = 0
        compteur = 0
        while s < q: 
            if np.sum((value-cU[s])**2, axis = 1) >= RU[s]**2:
                compteur = compteur + 1
            s = s + 1
        if compteur == q:
            myReturn[0] == False
        
    return myReturn

* * *

### An example

In [ ]:
epsilon = 10
diff = 1
#n = nb of varialbles
n = 2
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 2
(cI, RI, cU, RU) = dataGenerator(epsilon, n, p, q, diff)

In [ ]:
IsIncluded_PLOT2d(cI, RI, cU, RU, p, q)

In [ ]:
IsIncluded_QCQP(cI, RI, cU, RU, n, p, q)

In [ ]:
IsIncluded_QCQP(cI, RI, cU, RU, n, p, q, q-1)

In [ ]:
IsIncluded_QP(cI, RI, cU, RU, n, p, q)

In [ ]:
plot2d(cI, RI, cU, RU, p, q)

<img src="http://media.giphy.com/media/3o6Mb3Feec33LawNdm/giphy.gif"  width="300">

<a id="3"></a> 
 
# 3. QP vs. QCQP

## Exactness

In [ ]:
#n = nb of variables
n = 2
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 2
epsilon = 10
diff = 1
nbQCQP = 0
nbQCQPbis = 0
nbQP = 0
nbTrue = 0

print("ITERATION", "nbTrue", "# QCQP = exactSol","# QCQP(q-1) = exactSol" , "# QP = exactSol")
for i in range(0,10):
    (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
    Response = IsIncluded_PLOT2d(cI,RI,cU,RU,p,q)
    nbTrue = nbTrue + int(Response)
    
    resp1 = IsIncluded_QCQP(cI,RI,cU,RU,n,p,q)
    resp1BIS = IsIncluded_QCQP(cI,RI,cU,RU,n,p,q,q-1)      
    resp2 = IsIncluded_QP(cI,RI,cU,RU,n,p,q)
    
    nbQCQP = nbQCQP + int(resp1[0] == Response) 
    nbQCQPbis = nbQCQPbis + int(resp1BIS[0] == Response)          
    nbQP = nbQP + int(resp2[0] == Response) 
    
    print(i+1,nbTrue,nbQCQP,nbQCQPbis,nbQP)
    
    if(Response != resp2[0]):
        plot2d(cI,RI,cU,RU,p,q)
        break

#### TABLE 1 OF THE PAPER : dimension 2

In [ ]:
#n = nb of variables
n = 2
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 3
epsilon = 10
diff = 1
nbQCQP = 0
nbQCQPbis = 0
nbQP = 0

print("ITERATION", "nb Simu to get False", "# QCQP = exactSol","# QCQP(q-1) = exactSol", "# QP = exactSol")

for i in range(0,100):
    (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
    j = 1
    Response = IsIncluded_PLOT2d(cI,RI,cU,RU,p,q)
    while Response == True:
        (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
        Response = IsIncluded_PLOT2d(cI,RI,cU,RU,p,q)
        j = j+1
        
    nbQCQP = nbQCQP + int(IsIncluded_QCQP(cI,RI,cU,RU,n,p,q)[0] == Response)   
    nbQCQPbis = nbQCQPbis + int(IsIncluded_QCQP(cI,RI,cU,RU,n,p,q,q-1)[0]   == Response) 
    nbQP = nbQP + int(IsIncluded_QP(cI,RI,cU,RU,n,p,q)[0] == Response)          
    print(i+1,j, nbQCQP, nbQCQPbis, nbQP)

#### TABLE 1 OF THE PAPER : dimension > 2

In [ ]:
#n = nb of variables
n = 3
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 3
diff = 1
epsilon = 10
nbQCQP = 0
nbQCQPbis = 0
nbQP = 0
nbTrue = 0

print("ITERATION", "nb Simu to get False", "# QCQP = QP","# QCQP(q-1) = QP", "# QP = False")

for i in range(0,100):
    (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
    j = 1
    Response = IsIncluded_QP(cI,RI,cU,RU,n,p,q)
    while Response[0] == True:
        (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
        Response = IsIncluded_QP(cI,RI,cU,RU,n,p,q)
        j = j+1
    
    nbQCQP = nbQCQP + (1-int(IsIncluded_QCQP(cI,RI,cU,RU,n,p,q)[0]))
    nbQCQPbis = nbQCQPbis + (1-int(IsIncluded_QCQP(cI,RI,cU,RU,n,p,q,q-1)[0]))  
    nbQP = nbQP + (1-int(Response[0]))         
    print(i+1,j,nbQCQP,nbQCQPbis,nbQP)

## computational time

#### Last figure of the paper

In [ ]:
import time
n = 5
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 3
diff = 1
epsilon = 10

### Comparison over 1 example (table 2 of the paper)

In [ ]:
time_QP = 0
time_QCQP = 0
temp_time_QP = 0
temp_time_QCQP = 0

for i in range(0,10):
    j = 1   
    (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
    Response = IsIncluded_QP(cI,RI,cU,RU,n,p,q)
    while Response[0] == True:
        (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
        Response = IsIncluded_QP(cI,RI,cU,RU,n,p,q)
        j = j + 1
    
    time_startQP = time.clock()
    IsIncluded_QP(cI,RI,cU,RU,n,p,q)
    temp_time_QP = (time.clock() - time_startQP)
    time_QP = time_QP + temp_time_QP

    time_startQCQP = time.clock()
    Response = IsIncluded_QCQP(cI,RI,cU,RU,n,p,q)   
    temp_time_QCQP = (time.clock() - time_startQCQP)
    time_QCQP = time_QCQP + temp_time_QCQP
    print(i, j, temp_time_QP, temp_time_QCQP, Response[0], Response[1])

print(time_QP,time_QCQP)
print(time_QCQP/time_QP)

### Mean time with respect to dimension n

In [ ]:
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 3
diff = 1
epsilon = 10
nbQCQP = 0
nbQCQPbis = 0
nbQP = 0
nbTrue = 0
nbloops = 0
nbVertex = 0
print("ITERATION", "time_QP vs dimension")

res = list()
u = 10
for n in range(10, 2000, 10):
    j = 1
    time_QP = 0
    nbloops = 0
    nbVertex = 0
    for i in range(0,u):
        (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
        Response = IsIncluded_QP(cI,RI,cU,RU,n,p,q)    
        while Response == True:
            (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
            Response = IsIncluded_QP(cI,RI,cU,RU,n,p,q)
            j = j + 1
        
        time_startQP = time.clock()
        Response = IsIncluded_QP(cI,RI,cU,RU,n,p,q)      
        time_QP = time_QP + (time.clock() - time_startQP)   
        nbloops = nbloops + Response[1]
        nbVertex = nbVertex + Response[2]
    res.append(time_QP/u) #save for plotting
    print(n,j,time_QP/u, nbloops, nbVertex)
print("THE END")

In [ ]:
res

<img src="http://media.giphy.com/media/3o6Mb3Feec33LawNdm/giphy.gif"  width="300">
